## Příprava 
Neprve si data stáhneme a podíváme se, co je uvnitř. Data jsou uložena v knihovním formátu MARC s koncovnkou .mrc. 
Pro práci budeme používat knihovnu pymarc, která dokáže marcový formát jednodušše načíst.   
Nejprve si vypíšeme, jak takový záznam vypadá.  

In [1]:
from pymarc import MARCReader

# Cesta k marcovemu dokumentu
database = 'data/ucla/ucla_cle.mrc'

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Iterace skrz nacteny soubor
    for record in reader:
        # Vypsani marcovych zaznamu
        print(record)

        # Chceme vypsat pouze jeden zaznma, tak po prvnim vypsani zavolame funkci break
        break


=LDR  00875nab a22003134a 4500
=001  001457852
=003  CZ\PrUCL
=005  20220321104606.0
=008  090923e197808\\xr\\\\\\\\\\\\|||\||cze\d
=035  \\$a(ISIS-B70-MFN)78971
=035  \\$a(ISIS-B70-ID)785001
=040  \\$aABB060$bcze
=080  \\$a7.01$2MRF
=080  \\$a(0:82-4)$2MRF
=100  1\$aKliment, Alexandr,$d1929-2017$7jk01060558$4aut
=245  10$aKultura je když... /$cAlexandr Kliment.
=500  \\$aPodnázev zdrojového dokumentu: (London, Index on Censorship).
=520  2\$aÚvaha.
=599  \\$aCLB-CPK
=650  07$aestetika$7ph117189$2czenas
=655  \7$aúvahy$7fd134000$2czenas
=773  0\$tSpektrum$gR. 1978, č. 1, srpen, s. 77-79$q1978:1<77$9197808
=964  \\$aB70
=964  \\$aSMZ
=964  \\$aCLE
=OWN  \\$aUCLA
=SIF  \\$afk
=910  \\$aABB060
=ZAZ  \\$d197808$s1978$z1$l77



Vidíme, že marcový soubor má jasnou strukuru. Má několik polí, která jsou označena zpravidla třemi číslicem, případně třemi písmeny. Každý kód má svou vnitřní logiku, např. pole pro věcné popisy vždy začínají číslicí 6XX.



In [7]:
# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Iterace skrz nacteny soubor
    for record in reader:
        # Vypsani marcovych zaznamu
        # K nekterym zaznamum muzeme pristupovat pres teckovou notaci, tedy record.leader nebo record.title 
        print("Záznam: " + record.leader)
        
        # Pred vypsanim je standardem se nejprve podivat, zda zaznam existuje (tedy ze neni None). 
        # Pokud bychom se totiz snazili vypsat None zaznam, kod by vyhodil error. 
        if record.title is not None:
            print("Nazev: " + record.title)
        if record.author is not None:
            print("Autor: " + record.author)
        # Pokud chceme zjistit, zda pole ktere nema teckovou notaci neni None, je potreba zavolat funci .get_field(), pripadne .get_field()     
        if record.get_fields('655') is not None:    
            # K polim, ktere nemaji teckovou notaci, se pristupuje pres zavorky 
            print("Žánr: " + record['655']['a'])
        break        

Záznam: 01202nab a22003374a 4500
Nazev: Dělám si, co chci /
Autor: Radůza, 1973- jo2003184056 ive
Žánr: rozhovory


Také by nás mohlo zajímat, kolik záznamů je v dané databázi. K tomu si muzeme vytvorit samostatnou funkci, kterou pak jen zavolame

In [9]:
# Vlastni funkce definujeme pomoci slova def
def number_of_records(database):
    with open(database, 'rb') as data:
    # Nacteni marcu
        reader = MARCReader(data)
        # Vytvorime counter, ke kteremu pri kazdem zaznamu pricteme jednicku
        counter = 0
        for record in reader:
            counter += 1
    return counter 

print("V databazi je " + str(number_of_records(database)) + " zaznamu.")        

V databazi je 58934 zaznamu.


Teď je potřeba si marcová data uložit do jednodušší tabulky, se kterou později budeme pracovat. V této fázi si musíme ujasnit, jaká data budeme chtít zpracovat. V našem příkladu si budeme chtít uložit název, autora, autorův kód, rok vydání a pak pole '600','650' a '655'. 
Veškeré záznamy začínající číslem 6XX jsou věcné údaje o záznamu. Všechna pole se mohou opakovat.
Pod polem '600' se skrývají osoby, o kterých záznam je nebo případně osoby, kterým je záznam dedikován. 
Pod polem '650' se nacházejí věcné termíny/téma, tzn. o čem záznam je.
Pod polem '655' pak najdeme žánr daného záznamu. Na rozdíl od polí '600' a '650' by pole '655' mělo být přítomné u každého záznamu. 

K tomu jsme si napsali funkci save_to_dict(record, dict, field_list), která nám jeden záznam(record) uloží do slovníku dict. Protože si nebudeme chtít uložit všechny pole v záznamu, předáme funkci save_to_dict také seznam polí field_list, která budeme chtít uložit. Seznam field_list sestává z tuplů (kolekce v jazyce Python), kde každý tuple vypadá následovně. Na první pozici je název klíče, pod kterým bude pole uloženo, na druhé pozici tag pole a na třetí tag podpole, např. ('author', '100', 'a'). 

In [ ]:
import pandas as pd


def save_to_dict(record, dict, field_list):
    if not record is None:
        try:
            # Iterace skrz tuples v seznamu field_list
            for field_tags in field_list:
                # Nazev klice ve slovniku
                dict_key_name =  field_tags[0]

                # Tag pole
                tag =  field_tags[1]

                # Tag podpole
                subfield_tag =  field_tags[2]
                
                # Seznam do ktereho pridame hodnoty a nasledne pridame do slovniku
                dict_add_list = []
                
                # Iterace pres vsechna pole s tagem 'tag'
                for field in record.get_fields(tag):
                    
                    # Pokud pole nema zadna podpole, pridame cele pole do listu dict_add_list
                    if subfield_tag is None:
                        dict_add_list.append(str(field))
                    
                    # Pokud subtag je instance slice, tedy to znamena, ze chceme jen nejakou cast pole, ktera neni definovana subpolem,
                    # pridame cast pole do slovniku dict_add_list    
                    elif isinstance(subfield_tag, slice):
                        dict_add_list.append(str(field) [subfield_tag])     
                    
                    # Pokud pole obsahuje podpole, pridame do slovniku dict_add_list jen podpole
                    elif '$'+subfield_tag in str(field):  
                        dict_add_list.append(str(field[subfield_tag]))

                # Do klice z tuplu pridame cely seznam dict_add_list         
                dict[dict_key_name].append(dict_add_list)
        except Exception as error:
            print("Exception: " + type(error).__name__)  
            print("964 Field: " + str(record.get_fields('964')))  
            print("LDR: " + str(record.leader))   
    return dict 
out = 'data/out.csv'

with open(database, 'rb') as data:
    reader = MARCReader(data)
    field_list = [('title', '245', 'a'),
                ('author', '100', 'a'),
                ('author code', '100', '7'),
                ('year', '008', slice(13,17, None)),
                ('figures', '600', 'a'),
                ('description', '650', 'a'),
                ('genre', '655', 'a')]
    dict = {}
    for t in field_list:
        dict_key_name = t[0]
        dict[dict_key_name] = []
    for record in reader:
        dict = save_to_dict(record, dict, field_list)
    df = pd.DataFrame.from_dict(dict)
    df['figures'] = df['figures'].apply(lambda x: [y[:-1] if isinstance(y, str) and len(y) > 0 else y for y in x]) 
    df['author'] = df['author'].apply(lambda x: [y[:-1] if isinstance(y, str) and len(y) > 0 else y for y in x])  
    for column in df.columns:
        df[column] = df[column].apply(lambda x: ';'.join(x))
    df.to_csv(out, encoding = 'utf8', sep = ",")     